In [1]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np
import random

In [2]:
SiO = Structure.from_file("benchmarking/SiO2.cif")
SiO.make_supercell([[24, 0, 0], [0, 30, 0], [0, 0, 30]])
SiO_atoms = AseAtomsAdaptor().get_atoms(SiO)
len(SiO_atoms)

/data/kevinxhan/miniconda3/envs/distmlip/lib/python3.10/site-packages/pymatgen/core/structure.py:3100: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


194400

In [3]:
import matgl
from DistMLIP.implementations.matgl import Potential_Dist, PESCalculator_Dist
from DistMLIP.implementations.matgl import CHGNet_Dist, TensorNet_Dist
from DistMLIP.implementations.matgl import MolecularDynamics, Relaxer


model = matgl.load_model('CHGNet-MPtrj-2023.12.1-2.7M-PES').model
dist_model = CHGNet_Dist.from_existing(model).eval()
dist_model.enable_distributed_mode([0, 1, 2, 3, 4, 5, 6, 7])

/data/kevinxhan/miniconda3/envs/distmlip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pes = Potential_Dist(model=dist_model, num_threads=128)
pes(SiO_atoms)

Cartesian maxes: 117.959188 126.105750 162.296922
Cartesian mins: 0.000000 -72.283816 0.000000
Fractional longest dim max: 0.995194
Fractional longest dim min: 0.000000
Longest dim length in fractional coordinates: 0.995194
Checking collisions... 
Dim max: 0.995194. Dim min: 0.000000
First wall position: 0.124399
Bond graph is enabled but atom_cutoff is 6.000000 and bond_cutoff is 3.000000. The partition width is 15.885113 which is <= 2 * (atom_cutoff + bond_cutoff) (18.000000).
A wall width that is <= 2 * (atom_cutoff + bond_cutoff) will be inefficient but still yield correct results.
You should reduce the # of partitions. If you cannot fit your system on a reduced # of partitions, then your system is probably too dense.
Contact Kevin (kevinhan@cmu.edu) if you need help with this.
Created partition rule
Dim to use (based on cartesian coordinates): 1
Atom cutoff: 6.000000
Number of bonds within atom radius: 14256000
Number of bonds within bond radius: 1296000
USE_BOND_GRAPH IS 1
BDE gl

OutOfMemoryError: CUDA out of memory. Tried to allocate 508.00 MiB. GPU 0 has a total capacty of 79.14 GiB of which 74.75 MiB is free. Including non-PyTorch memory, this process has 79.05 GiB memory in use. Of the allocated memory 75.80 GiB is allocated by PyTorch, and 2.77 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
positions = SiO_atoms.get_positions()
cell = SiO_atoms.get_cell()
pbc = np.array(SiO_atoms.get_pbc(), dtype=np.int64)
num_partitions = 8
cutoff = 6

In [ ]:
from DistMLIP.distributed.dist import Distributed

dist = Distributed.create_distributed(
        cart_coords = positions,
        frac_coords = Distributed.cartesian_to_wrapped_fractional(positions, cell, pbc),
        lattice_matrix = cell,
        num_partitions = num_partitions,
        pbc = pbc,
        cutoff = cutoff,
        num_threads = 1 # TODO: change this later to environment variable
    )